Note: This is a toy example on a small simulated data set. These parameters were optimized for real and larger simulated datasets. 

In [1]:
# Set number of threads to use
import os
nthreads = 8
os.environ["MKL_NUM_THREADS"] = str(nthreads)
os.environ["NUMEXPR_NUM_THREADS"] = str(nthreads)
os.environ["OMP_NUM_THREADS"] = str(nthreads)

In [2]:
import sys

In [3]:
sys.path.insert(1, '../CoreFunctions/')

In [4]:
from JSTA import *

# Load the simulated data
spots: spatial transcriptomics data
nuclei: coordinates of the nuclei pixels, with id numbers

In [5]:
pref = '../simulated_example/'
# load spots
with open(pref+'spots.npy', 'rb') as f:
    spots = np.load(f, allow_pickle=True)
    spots = pd.DataFrame(spots)
    spots.columns = ['x', 'y', 'z', 'gene']
    spots = spots.loc[:, ['gene', 'x', 'y', 'z']]
# load nuclei
with open(pref+'nuclei.npy', 'rb') as f:
    nuclei = np.load(f, allow_pickle=True)
    nuclei = pd.DataFrame(nuclei)
    nuclei.columns = ['id', 'x', 'y', 'z']


Ensure the nuclei start at 0 and end at the number of nuclei - 1  
Some nuclei get filtered out during simulation, and the ids are not updated.  
This may also be the case during preprocessing and nuclei segmentation in real data  

In [6]:
for i, nuc_id in enumerate(np.unique(nuclei.id)):
    nuclei.loc[nuclei.id == nuc_id, 'id'] = i

# Read in reference datasets

In [7]:
pref = '../ref_data/'
sc_ref = pd.read_csv(pref+'sc_ref.csv.gz', index_col=0)
sc_celltypes = pd.read_csv(pref+'celltypes.txt.gz',
                          header=None).to_numpy().ravel()

only keep genes in both reference and spatial transcriptomics dataset

In [8]:
all_genes = np.intersect1d(sc_ref.columns, np.unique(spots.gene))

In [9]:
#remove extra genes from spots and from reference
sc_ref = sc_ref.loc[:,all_genes]
spots = spots.loc[np.isin(spots.gene,all_genes),:]

In [10]:
# cell type map can be used later to go back to
# cell type names
cell_type_map = {}
for i, c in enumerate(np.unique(sc_celltypes)):
    cell_type_map[i] = c
    
    # change cell types to int for training
    sc_celltypes[sc_celltypes == c] = i

# required for tensorflow formatting
sc_celltypes = np.array(sc_celltypes, dtype=int)

# Subset the dataset to marker genes (Optional)
We train a random forest classifier on the whole reference to identify marker genes

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
clf_rf = RandomForestClassifier(n_estimators = 1000, n_jobs=nthreads).fit(sc_ref.values, sc_celltypes)

In [32]:
number_of_genes_to_keep = 82 # For now I selected all genes
subset_marker_genes = np.sort([i[1] for i in sorted(zip(clf_rf.feature_importances_, sc_ref.columns))][::-1][:number_of_genes_to_keep])

Subset the reference and spots to those genes

In [33]:
sc_ref = sc_ref.loc[:,subset_marker_genes]
spots = spots[np.isin(spots.gene, subset_marker_genes)]

# Run density estimation:
We use a KNN based density estimation to get the local density at each pixel:    
  
$$\frac{\textit{num_spots_around}}{\frac{4}{3}{\pi}r^{3}}$$  
  
pixel_length: edge length of each pixel in microns. (Lower is higher resolution)  
num_spots_around: Number of transcripts on which to find the volume

In [ ]:
num_spots_around = 5
pixel_length = 1
pixels = fast_de_all_spots(spots, pixel_length, num_spots_around)
locations = get_locations(spots, pixel_length)

Bin the count data into pixels

In [ ]:
pix_true = get_real_pixels(spots, pixel_length, all_genes, pixels.shape)

In [ ]:
# Plot the expression intensity
plt.imshow(np.log2(np.sum(pixels, axis=(2, 3))+1),
           cmap='inferno')
plt.axis('off')

# Initialize segmentation with watershed on distance transform
max_dist_to_nuclei: maximum distance from the edge of the nucleus ($\mu$) for a pixel to be assigned to a specific nucleus. We start conservative with a maximum radius of 3

In [ ]:
max_dist_to_nuclei = 3
cell_assignment = classify_pixels_to_nuclei(
    locations, nuclei, max_dist_to_nuclei)

In [ ]:
# note colors are randomized so they will change everytime
plot_segmentation(cell_assignment,
                  'nipy_spectral')

# Get the initialized counts matrix

In [ ]:
cells_mat = get_matrix_of_cells(pix_true, cell_assignment, nuclei)

# Train cell type classifier

In [ ]:
clf_cell = create_celltype_classifier(sc_ref, sc_celltypes,
                                      nlayers=2, l1_reg=5e-3,
                                      epochs=20, lrs=[5e-3, 5e-4],
                                      test_size=0.25)

In [ ]:
tic = time()
cell_assign, counts_mat, cell_types = reclassify_squares(pixels, pix_true,
                                                         cells_mat, nuclei,
                                                         cell_assignment,
                                                         sc_ref, sc_celltypes,
                                                         all_genes, locations,
                                                         clf_cell,
                                                         pct_train=0.1, border_other_threshold=5,
                                                         border_same_threshold=2,
                                                         outer_max=3, inner_max=5,
                                                         most_inner_max=5, dist_threshold=2, dist_scaling=5,
                                                         anneal_param=0.05, flip_thresh=0.2,
                                                         nlayer=3, first_epochs=25, second_epochs=15,
                                                         lrs=[1e-3, 1e-4], l1_reg=1e-3)

toc = time()
print('time for segmentation:', toc-tic)

In [ ]:
plot_segmentation(cell_assign)

# Map cell types back to original names

In [ ]:
real_celltypes = []
for i in cell_types:
    real_celltypes.append(cell_type_map[i])
real_celltypes = np.array(real_celltypes)